# LAMMPS Tutorials 01. Running your first LAMMPS simulation!

### Author: Mark Tschopp, mark.a.tschopp.civ at mail.mil

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/mrkllntschpp/lammps-tutorials.

The original tutorials are given here: https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_tutorials.  A number of these tutorials are out of date and have been ported over into the current iPython Jupyter Notebook tutorials on github.

***

## Description:
<a id="Sec1"></a>
This is a quick tutorial to running a LAMMPS simulation on a Windows machine. For this simple example, the molecular simulation calculates the equilibrium lattice constant and corresponding cohesive energy for aluminum in the face-centered cubic (fcc) lattice structure (see below).

<img src="https://icme.hpc.msstate.edu/mediawiki/images/e/ef/Fcc_stereo.gif" alt="Face-centered Cubic Lattice Structure" title="FCC Lattice Structure" />

***
## Step 1: Download LAMMPS for Windows Executable
<a id="Step1"></a>

Follow these steps to download the LAMMPS Windows Executable: 
1. Goto the LAMMPS download webpage, https://lammps.sandia.gov/download.html
1. Click on Pre-built LAMMPS Windows executables 
1. Click the "download now" button. 

If you wish to run LAMMPS in a Unix shell, you must download a version on the download webpage and compile an executable. 

Voila! Easy as that. 


***
## Step 2: Download an Interatomic Potential
<a id="Step2"></a>

Follow these steps to download an interatomic potential. 
1. Go to the NIST Interatomic Potential webpage, https://www.ctcms.nist.gov/potentials/
1. Click on Al for an aluminum single element potential 
1. Click on whichever potential you like. For this example, use the Mishin *et al.* potential, Al99.eam.alloy 
1. Save this in the same directory as the LAMMPS executable. 

Go ahead and open up the file in Wordpad, if you want to see what an interatomic potential for Al in the embedded atom method (EAM) alloy format looks like.  There is usually header information pertaining to the author of the potential and the first peer-reviewed paper describing its use.

**Potential Mistake 1** 

If you are not careful when you download, Windows may add a .txt or .html to the end of the file.  Then when the LAMMPS input script tries to call the file, there is an error because it can't find it.

**Potential Mistake 2** 

If it is not saved in the same directory where LAMMPS is run out of, LAMMPS will not know where to find it.

***
## Step 3: Download an Input File
<a id="Step3"></a>

This input script was run using the Aug 2015 version of LAMMPS. Changes in some commands may require revision of the input script. To get the input file, you have a few options:

*  Copy the text below and paste it into a text file, `calc_fcc.in`. Use the `Paste Special` command with unformatted text into the file. 
*  Or, I added the command `%%writefile calc_fcc.in` to the Jupyter Notebook which should just do everything for you!

Obviously, if the second option worked, you are all set!

In [1]:
%%writefile calc_fcc.in

# Find minimum energy fcc (face-centered cubic) atomic configuration
# Mark Tschopp

# ---------- Initialize Simulation --------------------- 
clear 
units metal 
dimension 3 
boundary p p p 
atom_style atomic 
atom_modify map array

# ---------- Create Atoms --------------------- 
lattice fcc 4
region box block 0 1 0 1 0 1 units lattice
create_box 1 box

lattice fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1  
create_atoms 1 box
replicate 1 1 1

# ---------- Define Interatomic Potential --------------------- 
pair_style eam/alloy 
pair_coeff * * Al99.eam.alloy Al
neighbor 2.0 bin 
neigh_modify delay 10 check yes 
 
# ---------- Define Settings --------------------- 
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 

# ---------- Run Minimization --------------------- 
reset_timestep 0 
fix 1 all box/relax iso 0.0 vmax 0.001
thermo 10 
thermo_style custom step pe lx ly lz press c_eatoms 
min_style cg 
minimize 1e-25 1e-25 5000 10000 

variable natoms equal "count(all)" 
variable teng equal "c_eatoms"
variable length equal "lx"
variable ecoh equal "v_teng/v_natoms"

print "Total energy (eV) = ${teng};"
print "Number of atoms = ${natoms};"
print "Lattice constant (Angstoms) = ${length};"
print "Cohesive energy (eV) = ${ecoh};"

print "All done!"

Overwriting calc_fcc.in


Awesome!  That little script should have written the above text to the file `calc_fcc.in`.  To check, let's execute a command on the present directory listing all files that end in `*.in`.

In [2]:
!dir *.in

 Volume in drive C is Windows
 Volume Serial Number is E645-9FFB

 Directory of C:\Users\mark.a.tschopp\Jupyter-Notebook\LAMMPSNotebooks

04/03/2020  03:45 PM             1,331 calc_fcc.in
04/03/2020  03:43 PM             1,463 calc_fcc_ver1.in
04/03/2020  03:43 PM             1,179 calc_fcc_ver2.in
               3 File(s)          3,973 bytes
               0 Dir(s)  71,857,131,520 bytes free


If that didn't seem to work, we could open a (new) input file to write this script to it using Python (I removed all the comments):

In [3]:
file = open("calc_fcc.in", "w")
file.write("clear\n ")
file.write("units metal \n ")
file.write("dimension 3 \n ")
file.write("boundary p p p \n ")
file.write("atom_style atomic \n ")
file.write("atom_modify map array \n ") 
file.write("lattice fcc 4 \n ")
file.write("region box block 0 1 0 1 0 1 units lattice \n ")
file.write("create_box 1 box \n ")
file.write("lattice fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1\n ")
file.write("create_atoms 1 box \n ")
file.write("replicate 1 1 1\n ")
file.write("pair_style eam/alloy\n ") 
file.write("pair_coeff * * Al99.eam.alloy Al\n ")
file.write("neighbor 2.0 bin \n ")
file.write("neigh_modify delay 10 check yes\n ")
file.write("compute eng all pe/atom \n ")
file.write("compute eatoms all reduce sum c_eng \n ")
file.write("reset_timestep 0 \n ")
file.write("fix 1 all box/relax iso 0.0 vmax 0.001\n ")
file.write("thermo 10 \n ")
file.write("thermo_style custom step pe lx ly lz press c_eatoms \n ")
file.write("min_style cg \n ")
file.write("minimize 1e-25 1e-25 5000 10000 \n ")
file.write("variable natoms equal \"count(all)\"\n ") 
file.write("variable teng equal \"c_eatoms\"\n ")
file.write("variable length equal \"lx\"\n ")
file.write("variable ecoh equal \"v_teng/v_natoms\"\n ")
file.write("print \"Total energy (eV) = ${teng};\"\n ")
file.write("print \"Number of atoms = ${natoms};\"\n ")
file.write("print \"Lattice constant (Angstoms) = ${length};\"\n ")
file.write("print \"Cohesive energy (eV) = ${ecoh};\"\n ")
file.write("print \"All done!\"\n ")
file.close()

***
## Step 4: Running LAMMPS in Windows

### Run this using LAMMPS in Windows, Method 1 

Follow these steps if running LAMMPS in Windows: 
1. Click on the Start button on the toolbar 
1. Click on Run... 
1. Enter `cmd` and hit OK 
1. In new window, change to the directory that contains the LAMMPS executable (lmp_win_no-mpi.exe), the input script (calc_fcc.in), and the potential file (Al99.eam.alloy). For example, if these are on the desktop, type 'cd h:/desktop' to change to the desktop (or wherever these are). 
1. Type `lmp_serial < calc_fcc.in` 
1. If the simulation completes, then it will type "All done!" Awesome!

Wait, what? It didn't work? OK, quick check. Can you actually run the LAMMPS executable?  Find the executable and its path.  Most likely the executable is in a different directory than the input script and the potential file.  So, try this:

In the command prompt window, can you type in the location of the LAMMPS executable?  If it's in the same directory that you are in, then it could be as easy as `lmp_serial`.  If not, you can run LAMMPS even when it lies in another directory somewhere, like in `C:/Program\ Files/LAMMPS\ 64-bit\ 20Apr2018/bin/lmp_serial`.  If you click enter and nothing happens, then ... good!  Hit `CTL + C` to exit.  It should tell you the current version, i.e., "LAMMPS (20 Apr 2018)".  If it gives you an error, then the path to the executable isn't right, try again.  Remember that if you are using folders with spaces in them, then you need to enter `'\' + ' '` within the path.

### Run this using LAMMPS in Windows, Method 2 

An alternative way to run this in Windows is to develop a _*.bat_ file, which is a Windows executable file. For instance, create a new text file and change the extension from *.txt* to *.bat*. By double clicking on a _*.bat_ file, you will execute the lines of scripts within the file. Now, don't double click to open, but rather right click and click on edit. Then enter the following lines: 

<div>
@echo off <br>
lmp_serial.exe < calc_fcc.in <br>
echo All done! <br>
echo Press a key to end <br>
pause > nul </div>

...which works if your administrator lets you run executables from your batch files on Windows 10. Try it by dropping it in the same folder as the LAMMPS executable, input script, and potential file and double clicking... voila!

### Run this using LAMMPS in Jupyter Notebook, Method 3

OK, this is cool!  We can actually run this from Jupyter Notebook.  Let's try it.

In [4]:
!"C:/Program Files/LAMMPS 64-bit 20Apr2018/bin/lmp_serial" < calc_fcc.in

LAMMPS (20 Apr 2018)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:90)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:90)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4 4 4
Created orthogonal box = (0 0 0) to (4 4 4)
  1 by 1 by 1 MPI processor grid
Lattice spacing in x,y,z = 4 4 4
Created 4 atoms
  Time spent = 0.00300002 secs
Replicating atoms ...
  orthogonal box = (0 0 0) to (4 4 4)
  1 by 1 by 1 MPI processor grid
  4 atoms
  Time spent = 0.000997066 secs
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.28721
  ghost atom cutoff = 8.28721
  binsize = 4.1436, bins = 1 1 1
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d/

Sweet!

### Run this using LAMMPS in UNIX, Method 4

If in a Unix environment, simply type `LAMMPS executable < input file`. If you wish to use multiple processors, use the mpirun command. For example, `mpirun -np 8 LAMMPS executable < input file` runs the simulation on 8 processors. 

The end of the logfile/screen output should look like this:

<div class="alert alert-block alert-info">
Total energy (eV) = -13.43999995; <br>
Number of atoms = 4;  <br>
Lattice constant (Angstoms) = 4.05;  <br>
Cohesive energy (eV) = -3.359999988;  <br>
All done! <br>
</div>

If you want to view the entire file (opening `log.lammps` in Notepad), then:

In [5]:
!start notepad log.lammps

***
## LAMMPS Input Script Explained

In this section, we will break down what LAMMPS is doing for each step. The easy way to do this on your own is to consult the LAMMPS manual for each command or go to the Internet LAMMPS manual, _i.e._, at https://lammps.sandia.gov

<div class="alert alert-block alert-info">
    # Find minimum energy fcc configuration  <br>
# Mark Tschopp </div>

The `#` denotes a comment. Hence LAMMPS does not do anything for these lines. 

<div class="alert alert-block alert-info">
# ---------- Initialize Simulation ---------------------  <br>
clear  <br>
units metal  <br>
dimension 3  <br>
boundary p p p  <br>
atom_style atomic  <br>
atom_modify map array <br>
</div>

This section initializes the simulations. The `clear` command clears all memory. The `units` command specifies the units that will be used for the remainder of the simulation; `metal` uses Angstroms and eV, among other units. The `dimension 3` command specifies a 3-dimensional simulation cell will be used (2-D is also an option). The `boundary p p p` specifies periodic boundaries in the x-, y-, and z-directions. 

<div class="alert alert-block alert-info">
# ---------- Create Atoms ---------------------  <br>
lattice 	fcc 4 <br>
region	box block 0 1 0 1 0 1 units lattice <br>
create_box	1 box <br> <br>
lattice	fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1   <br>
create_atoms 1 box <br>
replicate 1 1 1 <br></div>

The `lattice` command specifies what type of lattice is used (fcc, bcc, hcp, etc.) and the number following this specifies the lattice constant. The `region` command specifies the simulation cell. Here, we have used lattice units and specified that the simulation cell box is to be 1 lattice unit in each direction. The `create_box` command following this will use the parameters outlined in the `region` command to actually create the box. The `replicate` command can be used to replicate the periodic cell in each direction. 

<div class="alert alert-block alert-info">
# ---------- Define Interatomic Potential --------------------- <br>
pair_style eam/alloy  <br>
pair_coeff * * Al99.eam.alloy Al <br>
neighbor 2.0 bin  <br>
neigh_modify delay 10 check yes  <br>
    </div>

The `pair_style` command specifies what kind of interatomic potential will be used, while the `pair_coeff` specifies the file that the pair potential coefficients are stored in. The file extension for the interatomic potential file can sometimes give a clue as to which format is being used (eam.alloy = eam/alloy).

<div class="alert alert-block alert-info">
# ---------- Define Settings --------------------- <br>
compute eng all pe/atom <br>
compute eatoms all reduce sum c_eng <br>
</div>

Here, two computes are defined. In the first `compute` command, a variable named `eng` is defined to store the potential energy for each atom. In the second `compute` command, a variable named `eatoms` is defined to store the sum of all `eng` values. This is just to show how to use computes with user-defined variables.  Notice that the `pe` energy column during minimization is equivalent to the `c_eatoms` column, as expected.

<div class="alert alert-block alert-info">
# ---------- Run Minimization --------------------- <br> 
reset_timestep 0  <br>
fix 1 all box/relax iso 0.0 vmax 0.001 <br>
thermo 10  <br>
thermo_style custom step pe lx ly lz press c_eatoms  <br>
min_style cg  <br>
minimize 1e-25 1e-25 5000 10000   <br>
</div>

The `reset_timestep` does just that. The `fix` command uses the `box/relax` setting, whereby all directions (`iso`) are relaxed to 0.0 Pa pressure for all atoms (`all`). The `thermo` specifies the output during minimization. The `thermo_style` specifies what type of output is shown to screen. Here I have used a `custom` list of metrics, including the time`step`, the potential energy (`pe`), the length of the box in x, y, and z (`lx`, `ly`, `lz`, respectively), the `press`ure, and the computed variable `eatoms`. The `min_style` specifies that conjugate gradient will be used for minimization and the `minimize` starts the minimization, *i.e.*, the simulation cell boundaries are relaxed from the specified lattice constant (4 Angstroms) to the actual lattice constant (4.05 Angstroms). 

<div class="alert alert-block alert-info">
variable natoms equal "count(all)" <br>
variable teng equal "c_eatoms" <br>
variable length equal "lx" <br>
variable ecoh equal "v_teng/v_natoms" 
</div>

This section defines some variables, such as `natoms` for the number of atoms, `teng` for the total potential energy, `length` for the length of the simulation cell, and `ecoh` for the cohesive energy of Al. 

<div class="alert alert-block alert-info">
print "Total energy (eV) = \${teng};" <br>
print "Number of atoms = \${natoms};" <br>
print "Lattice constant (Angstoms) = \${length};" <br>
print "Cohesive energy (eV) = \${ecoh};" <br>
print "All done!"
</div>

This section prints these values to screen and to the logfile. The `${}` is used to insert the variables defined earlier. 

***
## FAQs 

</br>
<div class="alert alert-danger">
<strong>Question 1</strong>: I typed in the above line and nothing happened.
</div>

Make sure that you are in the same directory as the LAMMPS executable. Make sure that you are typing in the correct filename. 

<br>
<div class="alert alert-danger">
<strong>Question 2</strong>: I keep getting an error with the potential, i.e., potential file not found.
</div>

First, check that the potential file is in the directory that you are running out of. Although, you can insert paths, if you want to store the potentials in another directory. Second, make sure that the potential file name is the same as that given in the input script. For instance, Windows has the habit of saving text files (like the potential files) with .txt extensions. If you happen to be running from a Windows operating system, I would change the settings so that you can see the extensions of the files as well. 

***
## Tutorial Links

[Click here to open the next tutorial](LAMMPS-Tutorials-02.ipynb)